In [71]:
import numpy as np
import pandas as pd
from datetime import date, datetime, timedelta 
import re    
import json    
import time
import sys
sys.path.append('../')
from analysts_tools.growth import *
from analysts_tools.frucap import *
from analysts_tools.Clusters_querys import *
from analysts_tools.whitelist_functions import *

from procurement_lib import send_slack_notification
from procurement_lib import DataWarehouse, GoogleSheet
from analystcommunity import frubana_logger as logging
import logging as pylogging
import warnings
import pytz
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import ngrams
from nltk.stem import WordNetLemmatizer
from nltk import wordnet
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
import nltk
from collections import Counter
from itertools import chain
import itertools
from itertools import repeat
import math
from procurement_lib import redash

dw = DataWarehouse()
#warnings.filterwarnings("ignore")

from procurement_lib import redash
from analysts_tools.redash_methods import *

# Parameters and load queries

In [72]:
#//---------------------------------------------------------------------------------------------------------------------------
#// Parametros
todays_date = '2022-07-31'  
customer_date_desde = '2022-07-01' #3 months

In [73]:
query = """
    WITH 
    count_fulfillments_pasados AS (
        SELECT 
            count(distinct boi.order_item_id) as count_items,
            bfg.order_id
        FROM postgres_broadleaf_federate."broadleaf.blc_fulfillment_group"              bfg
        LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order"                     bo      ON bo.order_id = bfg.order_id
        LEFT JOIN postgres_broadleaf_federate."broadleaf.fb_fulfillment_group"          ffg     ON bfg.fulfillment_group_id = ffg.fulfillment_group_id
        LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_order"         bfo     ON bfg.fulfillment_group_id = bfo.fulfillment_group_id
        LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_payment"             bop     ON bop.order_id = bo.order_id
        LEFT JOIN postgres_broadleaf_federate."broadleaf.fb_order"                      fo      ON fo.order_id = bo.order_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                     s       ON bo.site_disc = s.site_id
        LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_order_type"                fot     ON fot.fb_order_type_id=fo.fb_order_type_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_group_item"   bfgi    ON bfgi.fulfillment_group_id = bfg.fulfillment_group_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_item"               boi     ON bfgi.order_item_id= boi.order_item_id
        
        WHERE      
            ffg.close_date >= '{desde}' AND ffg.close_date <= '{hasta}'
            AND bo.order_status='SUBMITTED'
            AND bfo.status NOT IN ('ARCHIVED','CANCELLED')
            AND fo.fb_order_status_id IN (1,6,7,8)
            AND bop.archived = 'N'
            AND (fot.name IS NULL OR fot.name <> 'REFUND')
        GROUP BY 2
    ),

    ful as (

        SELECT DISTINCT
            bo.order_id,
            ffg.close_date
            
        FROM postgres_broadleaf_federate."broadleaf.blc_order"                      bo
        INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_group"    bfg ON bfg.order_id = bo.order_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_order"    bfo ON bfo.fulfillment_group_id = bfg.fulfillment_group_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.fb_fulfillment_group"     ffg ON ffg.fulfillment_group_id = bfg.fulfillment_group_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order"                 fo  ON fo.order_id = bo.order_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_payment"        bop ON bop.order_id = bo.order_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                 s   ON s.site_id = bo.site_disc
        LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_order_type"            fot ON fot.fb_order_type_id=fo.fb_order_type_id
        LEFT JOIN  postgres_broadleaf_federate."broadleaf.blc_order_adjustment"     ba  ON ba.order_id = bo.order_id
        
        WHERE fo.fb_order_status_id IN (1,6,7,8)
            AND ffg.close_date >= '{desde}' AND ffg.close_date <= '{hasta}'
            AND bo.order_status = 'SUBMITTED'
            AND bfo.status NOT IN ('ARCHIVED','CANCELLED')
            AND bop.archived = 'N'
            AND (fot.name IS NULL OR fot.name <> 'REFUND')
            
        GROUP BY bo.order_id, ffg.close_date
    )--,

    --all_data AS (

    SELECT DISTINCT
        s.site_identifier_value as region_code,
        bcat2.name as cat,
        bcat.name as subcat,
        bs.sku_id,
        boi.name AS product,
        bo.customer_id,
        --count(distinct bo.customer_id) as penetracion,
        CASE 
            WHEN s.site_identifier_value = 'CMX' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + SUM((coalesce(bo.total_shipping,0) / cf.count_items)) + coalesce(SUM(foi.total_tax_iva),0))*1.0/19.19)
            WHEN s.site_identifier_value = 'GDL' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + SUM((coalesce(bo.total_shipping,0) / cf.count_items)) + coalesce(SUM(foi.total_tax_iva),0))*1.0/19.19)
            WHEN s.site_identifier_value = 'PBC' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + SUM((coalesce(bo.total_shipping,0) / cf.count_items)) + coalesce(SUM(foi.total_tax_iva),0))*1.0/19.19)
            WHEN s.site_identifier_value = 'MTY' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + SUM((coalesce(bo.total_shipping,0) / cf.count_items)) + coalesce(SUM(foi.total_tax_iva),0))*1.0/19.19)
            WHEN s.site_identifier_value = 'SPO' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + SUM((coalesce(bo.total_shipping,0) / cf.count_items)) + coalesce(SUM(foi.total_tax_iva),0))*1.0/3.88)
            WHEN s.site_identifier_value = 'BHZ' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + SUM((coalesce(bo.total_shipping,0) / cf.count_items)) + coalesce(SUM(foi.total_tax_iva),0))*1.0/3.88)
            WHEN s.site_identifier_value = 'CWB' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + SUM((coalesce(bo.total_shipping,0) / cf.count_items)) + coalesce(SUM(foi.total_tax_iva),0))*1.0/3.88)
            WHEN s.site_identifier_value = 'VCP' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + SUM((coalesce(bo.total_shipping,0) / cf.count_items)) + coalesce(SUM(foi.total_tax_iva),0))*1.0/3.88)
        ELSE ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + SUM((coalesce(bo.total_shipping,0) / cf.count_items)) + coalesce(SUM(foi.total_tax_iva),0))*1.0/3000) END AS net_gmv_usd

    FROM 
        postgres_broadleaf_federate."broadleaf.blc_fulfillment_group_item"                  bfgi
        INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_group"            bfg     ON bfgi.fulfillment_group_id = bfg.fulfillment_group_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.fb_fulfillment_group"             ffg     ON ffg.fulfillment_group_id = bfg.fulfillment_group_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order"                        bo      ON bo.order_id = bfg.order_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                         s       ON bo.site_disc = s.site_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_item"                   boi     ON bfgi.order_item_id= boi.order_item_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order_item"                    foi     ON boi.order_item_id= foi.order_item_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.fb_owner"                         fow     ON fow.owner_id = ffg.owner_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order"                         fo      ON fo.order_id = bo.order_id
        INNER JOIN ful                                                                              ON ful.order_id = bo.order_id
        LEFT JOIN count_fulfillments_pasados                                                cf      ON bfg.order_id = cf.order_id
        LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_order_type"                    fot     ON fot.fb_order_type_id=fo.fb_order_type_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.blc_discrete_order_item"          bdoi    ON bdoi.order_item_id = boi.order_item_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                          bs      ON bs.sku_id = bdoi.sku_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.blc_product"                      bp      ON bs.addl_product_id = bp.product_id
        left join postgres_broadleaf_federate."broadleaf.blc_category_xref"                 bcx     ON bcx.sub_category_id = bp.default_category_id and bcx.archived='N' and bcx.sndbx_tier is null and bcx.default_reference = 'true'
        left join postgres_broadleaf_federate."broadleaf.blc_category"                      bcat    ON bp.default_category_id = bcat.category_id
        left join postgres_broadleaf_federate."broadleaf.blc_category"                      bcat2   ON bcx.category_id = bcat2.category_id
        LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_price_dtl"          boipd   ON boipd.order_item_id=boi.order_item_id
        LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_dtl_adj"            boida   ON boida.order_item_price_dtl_id=boipd.order_item_price_dtl_id
        LEFT JOIN postgres_broadleaf_federate."broadleaf.fb_customer"              fc         ON fc.customer_id = bo.customer_id

    WHERE 
        ffg.close_date >= '{desde}' AND ffg.close_date <= '{hasta}'
        AND bo.order_status='SUBMITTED'
        AND fo.fb_order_status_id IN (1,6,7,8)
        AND (fot.name IS NULL OR fot.name <> 'REFUND')
        AND bcat2.name in ('Abarrotes','Aseo e Higiene','Bebidas','Congelados','Desechables','Lácteos & Huevos','Carne, Pollo & Pescados', 'Frutas & Verduras', 'Bebidas','Mercearia','Limpeza e Higiene','Laticínios e Ovos','Frutas e Verduras','Congelados','Descartáveis','Carnes, Aves e Peixes')
        AND fc.business_type_id IN (1)
        
    GROUP BY 1,2,3,4,5,6
""".format(desde=customer_date_desde, hasta=todays_date)
data_ventas = read_connection_data_warehouse.runQuery(query)

In [74]:
data_ventas.head()

,region_code,cat,subcat,sku_id,product,customer_id,net_gmv_usd
0,VCP,Mercearia,"Açúcar, Adoçantes e Doces",430828,Açúcar Alto Alegre 1Kg Pacote de 1Kg - A part...,180549220,82.345360824742268041237100
1,BOG,Frutas & Verduras,Frutas,144884,Plátano Verde Hartón Armenia (Semi) - Media C...,104300499,11.900000000000000000000000
2,GDL,Frutas & Verduras,Tubérculos,221015,Jicama Estándar - Kg,55602769,1.261073475768629494528400
3,BOG,Frutas & Verduras,Verduras,-303171,Yuca Estándar - Kg,60910517,9.477333333333333333333333
4,SPO,Bebidas,Refrigerantes e Energéticos,452044,Coca-Cola 2L Garrafa de 2l - A partir de 6 un...,185997826,12.974226804123711340206100


In [75]:
Query_SKU=get_fresh_query_result("https://internal-redash.federate.frubana.com/",97797,'9FFFsAzt85oV0jqIpdPolKrUJvjKTapU1Q89d1FD',{},2)

200


In [76]:
Query_MS=get_fresh_query_result("https://internal-redash.federate.frubana.com/",82561,'9FFFsAzt85oV0jqIpdPolKrUJvjKTapU1Q89d1FD',{},20)

200


In [77]:
regiones=data_ventas[['customer_id','region_code']].drop_duplicates()

In [78]:
regiones.head()

,customer_id,region_code
0,180549220,VCP
1,104300499,BOG
2,55602769,GDL
3,60910517,BOG
4,185997826,SPO


In [79]:
Query_MS_data=pd.merge(Query_MS,regiones,on='customer_id')

In [80]:
Query_MS_correct_MS=MS_corregidos_2(Query_MS_data)

## Create mind df

In [81]:
df_prev=pd.merge(data_ventas,Query_SKU, on = 'sku_id')
df=pd.merge(df_prev,Query_MS_correct_MS,how = 'left', on = 'customer_id')

In [82]:
df_top = df_prev.groupby(by=['region_code', 'cat', 'subcat', 'card_frida','card_name']).agg({'net_gmv_usd':np.sum, 'customer_id': 'nunique'}).reset_index()

## Sort in CSV the top 10 SKUs for each city

In [83]:
for city in df_top.region_code.unique():
     s = df_top[df_top['region_code']==city].sort_values(['customer_id'],ascending = False).head(10)
     s.to_csv(f'Analisis_{city}.csv', index = False)
    

# Begin with script to find top skus for each MS

In [84]:
cities=['BOG','BAQ','CMX','SPO']

df=df.rename(columns={'region_code_x':'region_code'})

df['microsegment']=df.apply(lambda x: x['microsegment'] if x['region_code'] in cities else 'Other',axis=1)
df['microsegment']=df['microsegment'].fillna('Other')

df = df[['region_code', 'category', 'subcategory', 'sku_id', 'card_frida', 'card_name', 'net_gmv_usd', 'customer_id', 'microsegment']].copy()

In [85]:
df.head()

,region_code,category,subcategory,sku_id,card_frida,card_name,net_gmv_usd,customer_id,microsegment
0,VCP,Mercearia,"Açúcar, Adoçantes e Doces",430828,325936,Açúcar Alto Alegre 1Kg Pacote de 1Kg A partir ...,82.345360824742268041237100,180549220,Other
1,VCP,Mercearia,"Açúcar, Adoçantes e Doces",430828,325936,Açúcar Alto Alegre 1Kg Pacote de 1Kg A partir ...,101.288659793814432989690700,150682554,Other
2,VCP,Mercearia,"Açúcar, Adoçantes e Doces",430828,325936,Açúcar Alto Alegre 1Kg Pacote de 1Kg A partir ...,9.149484536082474226804100,181047990,Other
3,VCP,Mercearia,"Açúcar, Adoçantes e Doces",430828,325936,Açúcar Alto Alegre 1Kg Pacote de 1Kg A partir ...,54.896907216494845360824700,162321887,Other
4,VCP,Mercearia,"Açúcar, Adoçantes e Doces",430828,325936,Açúcar Alto Alegre 1Kg Pacote de 1Kg A partir ...,18.298969072164948453608200,169966372,Other


In [86]:
def analizar_top_productos(data_ventas):
    """
    Está función recibe un dataframe con información de ventas (sale de la función data_ventas_query) y analiza, por sku, cuales son los 
    top productos.
    los criterios bajo el cual la función define top productos son 2: (1) penetración del sku_id (2) gmv_usd que mueve el sku_id.
    
    Argumentos:
    data_ventas: data de ventas que entrega la función data_ventas_query.
    
    Resultado:
    top_productos_df: dataframe ordenada con los top productos
    """
    def group_top_products(x):
        resultado = {}
        resultado["card_name"] = x["card_name"].unique()[0]
        resultado["category"] = x["category"].unique()[0]
        assert len(x["category"].unique()) == 1
        resultado["subcategory"] = x["subcategory"].unique()[0]
        assert len(x["subcategory"].unique()) == 1
        resultado["microsegment"] = x["microsegment"].unique()[0]
        assert len(x["microsegment"].unique()) == 1
        resultado["penetration"] = x["customer_id"].nunique()
        resultado["gmv_usd"] = x["net_gmv_usd"].sum()
        #assert len(x["sku_id"].unique()) == 1
        resultado["sku_id"] = '#'.join(x["sku_id"].unique())
        return pd.Series(resultado, index=list(resultado.keys()))
    
    data_ventas[["sku_id", "card_frida"]] = data_ventas[["sku_id", "card_frida"]].astype(str)

    top_productos_df = data_ventas.groupby(by=["card_frida"]).apply(group_top_products).reset_index()
    top_productos_df = top_productos_df.sort_values(by=["penetration", "gmv_usd"], ascending=False)
    num_customer_ids = data_ventas["customer_id"].nunique()
    gmv_total = data_ventas["net_gmv_usd"].sum()

    top_productos_df["penetration_porcentual"] = 100*top_productos_df["penetration"]/num_customer_ids
    top_productos_df["gmv_p_user"] = 100*top_productos_df["gmv_usd"]/num_customer_ids

    return top_productos_df

In [87]:
cities=['BOG','BAQ','MDE','CMX','GDL','PBC','SPO','BHZ','CWB','VCP']
DFDictCitySKU = {elem : df[df['region_code'] == elem] for elem in cities}

for i in cities:
    for j in DFDictCitySKU[i].microsegment.unique():
        filtro_ms = (DFDictCitySKU[i]['microsegment']==j).copy()
        BASE = DFDictCitySKU[i][filtro_ms]
        DFDictCitySKU[i+'_sku_'+j] = analizar_top_productos(BASE)
        DFDictCitySKU[i+'_sku_'+j]['region_code'] = i
        k=j.replace("/","_")
        #DFDictCitySKU[i+'_sku_'+j].to_csv(f'Top_skus_MS/top_{i}_skus_{k}.csv', index=False)

/opt/tljh/user/lib/python3.7/site-packages/pandas/core/frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [88]:
DFDictCitySKU['BOG_sku_Comida rápida'].columns

Index(['card_frida', 'card_name', 'category', 'subcategory', 'microsegment',
       'penetration', 'gmv_usd', 'sku_id', 'penetration_porcentual',
       'gmv_p_user', 'region_code'],
      dtype='object')

In [89]:
df_ms = pd.DataFrame(columns = ['card_frida', 'card_name', 'category', 'subcategory', 'microsegment', 'penetration', 'gmv_usd', 'sku_id', 
                                'penetration_porcentual', 'gmv_p_user', 'region_code'])

for i in cities:
    for j in DFDictCitySKU[i].microsegment.unique():
        df2 = DFDictCitySKU[i+'_sku_'+j]
        df_ms = pd.concat([df_ms, df2], ignore_index=True)

In [90]:
df_ms[['region_code', 'microsegment', 'category', 'subcategory', 'card_frida', 'card_name', 'gmv_usd', 
       'gmv_p_user', 'penetration', 'penetration_porcentual', 'sku_id']].to_excel(f'Analisis.xlsx', index = False)